Тут находится код для запуска моего оригинального бейзлайна на русском ворднете. 
Чтобы всё запустилось, нужно склонировать репозиторий, установить requirements.txt, и скачать какую-то модель word2vec, правильно прописав путь к ней в ячейке в следующем разделе. Параметрами кодирования текстов (например, весами POS и добавлением POS тегов при поиске слов) можно управлять в ячейках, где создаётся W2VWrapper

In [1]:
import pandas as pd
import numpy as np
import pickle
from collections import Counter, defaultdict

In [2]:
import sys
import os
def add_sys_path(p):
    p = os.path.abspath(p)
    if p not in sys.path:
        sys.path.append(p)

add_sys_path('..')

In [3]:
import evaluate

In [4]:
import data_split

In [5]:
from evaluate import read_dataset

In [6]:
import json

In [7]:
from tqdm.auto import tqdm, trange

In [45]:
from sklearn.neighbors import KDTree

# Загрузка моделей w2v

In [37]:
import my_knn
from importlib import reload
reload(my_knn)

<module 'my_knn' from '/home/dale/taxonomy-enrichment-en/taxonomy-enrichment/experiments/my_knn.py'>

In [68]:
w2v = gensim.models.KeyedVectors.load_word2vec_format(  #load_word2vec_format
    #'../../models/all.norm-sz100-w10-cb0-it1-min100.w2v',
    '../../models/tayga_upos_skipgram_300_2_2019/model.bin',
    #'C:/Users/ddale/Downloads/NLP/rusvectores/taiga_skipgram/model.bin', 
    binary=True,
    unicode_errors='ignore',
)

In [70]:
add_pos = True
n = 300

In [71]:
w2v_embedder = my_knn.W2VWrapper(w2v, n=n, add_pos=add_pos)

In [72]:
w2v_embedder_pos = my_knn.W2VWrapper(w2v,  n=n, add_pos=add_pos, pos_weights={'NOUN': 1.0, 'PREP': 0.1}, default_weight=0.5)

In [73]:
w2v_embedder_pos_vb = my_knn.W2VWrapper(w2v,  n=n, add_pos=add_pos, pos_weights={'VERB': 1.0, 'PREP': 0.1}, default_weight=0.5)

# Оценка применения

In [44]:
def w2v_scorer(text1, text2):
    return np.dot(w2v_embedder.get_text_vec(text1), w2v_embedder.get_text_vec(text2)) ** 5

In [41]:
public_test_verbs = pd.read_csv('../data/public_test/nouns_public.tsv', header=None)
public_test_verbs.columns = ['text']

In [42]:
full_syn_storage, full_rel_storage, full_rel_df = my_knn.prepare_storages(
    synsets_filename='../data/ruwordnet/synsets.N.xml',
    relations_filename='../data/ruwordnet/synset_relations.N.xml',
    forbidden_words=set()
)

number of texts: 86549
forbidden senses are 0
numer of ids 29296 long list is 119273



28118
555
5
37249


In [46]:
full_w2v_vecs = np.stack([w2v_embedder(t) for t in tqdm(full_syn_storage.texts_long) ])
full_w2v_tree = KDTree(full_w2v_vecs)

In [47]:
full_w2v_vecs_pos = np.stack([w2v_embedder_pos(t) for t in tqdm(full_syn_storage.texts_long) ])
full_w2v_tree_pos = KDTree(full_w2v_vecs_pos)

In [48]:
public_test_hypos = {
    txt: 
    my_knn.hypotheses_knn(
        txt, 
        #index=full_ft_tree, text2vec=ft_embedder, 
        index=full_w2v_tree_pos, text2vec=w2v_embedder_pos,
        synset_storage=full_syn_storage, rel_storage=full_rel_storage,
        decay=3, 
        k=100, 
        grand_mult=0.5,
        neighbor_scorer=w2v_scorer,
    )  
    for txt in tqdm(public_test_verbs.text)
}

In [49]:
sub = my_knn.dict2submission(public_test_hypos, syns_storage.id2synset)
sub

,noun,result,result_text
0,АБДОМИНОПЛАСТИКА,616-N,БОЛЕЗНЬ
1,АБДОМИНОПЛАСТИКА,3406-N,ПОВРЕЖДЕНИЕ ЗДОРОВЬЯ
2,АБДОМИНОПЛАСТИКА,6129-N,СЕРДЕЧНО-СОСУДИСТЫЕ ЗАБОЛЕВАНИЯ
3,АБДОМИНОПЛАСТИКА,106665-N,"НАРУШИТЬ СОСТОЯНИЕ, ХОД"
4,АБДОМИНОПЛАСТИКА,106567-N,ФИЗИЧЕСКОЕ САМОЧУВСТВИЕ
...,...,...,...
7615,ЯТАГАН,146854-N,КЛИНКОВОЕ ОРУЖИЕ
7616,ЯТАГАН,106553-N,"ПРИСПОСОБЛЕНИЕ, ИНСТРУМЕНТ"
7617,ЯТАГАН,145210-N,ЗАЩИТНОЕ СНАРЯЖЕНИЕ
7618,ЯТАГАН,153217-N,ОСТРАЯ СТОРОНА


In [50]:
sub.to_csv('results/nouns_v4_w2v_only_simp5_pnlty_pos_weight_pan_small.tsv', sep='\t', encoding='utf-8', header=None, index=None)
sub.head(15)

,noun,result,result_text
0,АБДОМИНОПЛАСТИКА,616-N,БОЛЕЗНЬ
1,АБДОМИНОПЛАСТИКА,3406-N,ПОВРЕЖДЕНИЕ ЗДОРОВЬЯ
2,АБДОМИНОПЛАСТИКА,6129-N,СЕРДЕЧНО-СОСУДИСТЫЕ ЗАБОЛЕВАНИЯ
3,АБДОМИНОПЛАСТИКА,106665-N,"НАРУШИТЬ СОСТОЯНИЕ, ХОД"
4,АБДОМИНОПЛАСТИКА,106567-N,ФИЗИЧЕСКОЕ САМОЧУВСТВИЕ
5,АБДОМИНОПЛАСТИКА,108862-N,ВОСПАЛЕНИЕ (БОЛЕЗНЕННЫЙ ПРОЦЕСС)
6,АБДОМИНОПЛАСТИКА,1063-N,МЕДИЦИНСКОЕ ОБСЛЕДОВАНИЕ
7,АБДОМИНОПЛАСТИКА,119425-N,"ВЯЛЫЙ (ЛИШЕННЫЙ БОДРОСТИ, ЭНЕРГИИ)"
8,АБДОМИНОПЛАСТИКА,123474-N,"СКРЫТЫЙ, НЕЯВНЫЙ"
9,АБДОМИНОПЛАСТИКА,8804-N,ПРОСТУДНОЕ ЗАБОЛЕВАНИЕ


```
map: 0.42400429113
mrr: 0.459982606341
```  - на маленьких векторах

### Паблик на глаголах

In [74]:
public_test_verbs = pd.read_csv('../data/public_test/verbs_public.tsv', header=None)
public_test_verbs.columns = ['text']

In [75]:
full_syn_storage_v, full_rel_storage_v, full_rel_df_v = my_knn.prepare_storages(
    synsets_filename='../data/ruwordnet/synsets.V.xml',
    relations_filename='../data/ruwordnet/synset_relations.V.xml',
    forbidden_words=set()
)

number of texts: 32822
forbidden senses are 0
numer of ids 7521 long list is 49002



7408
218
5
10317


In [76]:
full_w2v_vecs_pos_v = np.stack([w2v_embedder_pos_vb(t) for t in tqdm(full_syn_storage_v.texts_long) ])
full_w2v_tree_pos_v = KDTree(full_w2v_vecs_pos_v)

In [77]:
public_test_hypos = {
    txt: 
    my_knn.hypotheses_knn(
        txt, 
        #index=full_ft_tree, text2vec=ft_embedder, 
        index=full_w2v_tree_pos_v, text2vec=w2v_embedder_pos_vb,
        synset_storage=full_syn_storage_v, rel_storage=full_rel_storage_v,
        decay=3, 
        k=100, 
        grand_mult=0.5,
        neighbor_scorer=w2v_scorer,
    )  
    for txt in tqdm(public_test_verbs.text)
}

In [78]:
sub = my_knn.dict2submission(public_test_hypos, full_syn_storage_v.id2synset)
sub

,noun,result,result_text
0,АБСОЛЮТИЗИРОВАТЬ,116390-V,"ИСКАЗИТЬ, ПРЕДСТАВИТЬ В ЛОЖНОМ СВЕТЕ"
1,АБСОЛЮТИЗИРОВАТЬ,141697-V,ПРЕДСТАВИТЬ В ВИДЕ
2,АБСОЛЮТИЗИРОВАТЬ,149806-V,"УТВЕРЖДАТЬ, ДЕЛАТЬ УТВЕРЖДЕНИЕ"
3,АБСОЛЮТИЗИРОВАТЬ,106877-V,ВЫСКАЗАТЬ МНЕНИЕ
4,АБСОЛЮТИЗИРОВАТЬ,117532-V,НАСТОЯТЬ НА СВОЕМ МНЕНИИ
...,...,...,...
1745,ШТОПАТЬ,113291-V,СШИВАТЬ
1746,ШТОПАТЬ,110795-V,ПРИШИТЬ (ПРИКРЕПИТЬ)
1747,ШТОПАТЬ,110801-V,"ШИТЬЕ, ПОШИВ"
1748,ШТОПАТЬ,106493-V,СОЗДАТЬ (СДЕЛАТЬ СУЩЕСТВУЮЩИМ)


In [79]:
sub.to_csv('results/verbs_v4_w2v_only_simp5_pnlty_pos_weight_tayga.tsv', sep='\t', encoding='utf-8', header=None, index=None)
sub.head(15)

,noun,result,result_text
0,АБСОЛЮТИЗИРОВАТЬ,116390-V,"ИСКАЗИТЬ, ПРЕДСТАВИТЬ В ЛОЖНОМ СВЕТЕ"
1,АБСОЛЮТИЗИРОВАТЬ,141697-V,ПРЕДСТАВИТЬ В ВИДЕ
2,АБСОЛЮТИЗИРОВАТЬ,149806-V,"УТВЕРЖДАТЬ, ДЕЛАТЬ УТВЕРЖДЕНИЕ"
3,АБСОЛЮТИЗИРОВАТЬ,106877-V,ВЫСКАЗАТЬ МНЕНИЕ
4,АБСОЛЮТИЗИРОВАТЬ,117532-V,НАСТОЯТЬ НА СВОЕМ МНЕНИИ
5,АБСОЛЮТИЗИРОВАТЬ,106456-V,ВЫРАЗИТЬ (ПОКАЗАТЬ)
6,АБСОЛЮТИЗИРОВАТЬ,107692-V,"ДАТЬ ВОЗМОЖНОСТЬ, ПОЗВОЛИТЬ"
7,АБСОЛЮТИЗИРОВАТЬ,118698-V,"ДЕЙСТВИЕ, ЦЕЛЕНАПРАВЛЕННОЕ ДЕЙСТВИЕ"
8,АБСОЛЮТИЗИРОВАТЬ,119125-V,"РАСПРОСТРАНИТЬСЯ, ПОЛУЧИТЬ РАСПРОСТРАНЕНИЕ"
9,АБСОЛЮТИЗИРОВАТЬ,106710-V,"ОПРЕДЕЛИТЬ, ВЫЯСНИТЬ"
